## Install needed dependencies to load the LLaMAntino Anita model - just for test and fun :)

It will take some minutes

In [ ]:
!pip install python-dotenv
!pip install -U torch transformers
!pip install accelerate
!pip install bitsandbytes

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

## Load the LLaMAntino model

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

# model chosen is llamantino 3 (anita)
base_model = "swap-uniba/LLaMAntino-3-ANITA-8B-Inst-DPO-ITA"

# set config for quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

# set config model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
)

# load the model
tokenizer = AutoTokenizer.from_pretrained(base_model)
print(f'Model {base_model} loaded')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model swap-uniba/LLaMAntino-3-ANITA-8B-Inst-DPO-ITA loaded


## A very simple test prompt

In [ ]:
import transformers

# here we tell the model how to behave
sys = "Sei un an assistente AI per la lingua Italiana di nome LLaMAntino-3 ANITA " \
    "(Advanced Natural-based interaction for the ITAlian language)." \
    " Rispondi nella lingua usata per la domanda in modo chiaro, semplice ed esaustivo."

# here we set our question/request
question = "Chi è l'antagonista principale della fiaba Cappuccetto Rosso?"
question = 'Qual è il monte più alto in Italia?'

# structure of the messages
messages = [
    {"role": "system", "content": sys},
    {"role": "user", "content": question}
]

# object that actually handles the request
pipe = transformers.pipeline(
    model=model,                # use the model we have downloaded before - LLaMAntino-3-Anita
    tokenizer=tokenizer,        # use the tokenizer we have selected before
    return_full_text=False,     # langchain expects the full text
    task='text-generation',     # we want the model generates text
    max_new_tokens=256,         # max number of tokens to generate in the output
    temperature=0.1,            # temperature for more (1.0) or less (0.0) creative answers
    do_sample=True,
    top_p=0.9,
)

# answers of the model
sequences = pipe(messages)
model_response = ". ".join(seq['generated_text'] for seq in sequences)

print(model_response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Il monte più alto in Italia è il Monte Bianco di Courmayeur, Quattro Valli, Monté Cervin (in italiano noto come Monte Rosa, ma non deve essere confuso con la catena montuosa omonima) o Mont Cervin, in relazione alla località da cui si ammira, non ha un toponimo unico in italiano, ma le sue cime sono in Italia solo la minima, la Carro, 4,545 m, mentre le altre cime sono in Svizzera, è in realtà la Punta Dufour, con i suoi 4.634 metri, la cima più alta in territorio italiano, ma non in totale per l'Italia e il suo confinante, se considerato in questo senso, la cima più alta è la Pointe Einock, 4,675 m, in territorio non italiano ma sovrastante il confine, e visibile da un paesino italiano. 

In realtà, se si considera solo il territorio italiano, la cima più alta è la Punta Dufour, 4.634 m, ma se si considera la


## Let's now load some PDFs

In [ ]:
!pip install pypdf
!pip install tiktoken
!pip install langchain
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.7/987.7 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.0/372.0 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 8.4 MB/s eta 0:00:00


Let's download some documents from https://drive.google.com/file/d/1jrcWYeNU-na1UeSQMIOAOhvJP-UEIYgM/view?usp=share_link

In [ ]:
from langchain.document_loaders import PyPDFLoader

# these are the .pdfs we want to load
loaders = [
    PyPDFLoader("cappuccetto_rosso.pdf"),
    PyPDFLoader("hansel_gretel.pdf"),
    PyPDFLoader("lepre_tartaruga.pdf"),
    PyPDFLoader("doc.pdf")
]
pages = []
for loader in loaders:
    pages.extend(loader.load())

In [ ]:
from langchain.text_splitter import TokenTextSplitter

# split documents into pages
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=32)
docs = text_splitter.split_documents(pages)

# let's print the first page
print(f'Documents have been split into {len(docs)} pages; \
  here\'s the first one, that is from \
  {docs[0].metadata["source"]}:\n{docs[0].page_content}')



Documents have been split into 23 pages;   here's the first one, that is from   cappuccetto_rosso.pdf:
Cappuccetto Rosso! di Jakob e Wilhelm Grimm   C'era una volta una cara ragazzina; solo a vederla le volevan tutti bene, e specialmente la nonna, che non sapeva più cosa regalarle. Una volta le regalò un cappuccetto di velluto rosso, e, poiché le donava tanto ch'essa non volle più portare altro, la chiamarono sempre Cappuccetto Rosso. Un giorno sua madre le disse:  - Vieni, Cappuccetto Rosso, eccoti un pezzo di focaccia e una bottiglia di vino, portali alla nonna; è debole e malata e si ristorerà. Mettiti in via prima che faccia troppo caldo; e, quando sei fuori, va' da brava, senza uscir di strada; se no, cadi e rompi la bottiglia e la nonna resta a mani vuote. E quando entri nella sua stanza, non dimenticare di dir buon giorno invece di curiosare in tutti gli angoli. -Farò tutto per bene, - disse Cappuccetto Rosso alla mamma e le diede la mano.  Ma la nonna abitava fuori, nel bosco, 

In [ ]:
for x in range(len(docs)):
  print(docs[x].metadata)

{'source': 'cappuccetto_rosso.pdf', 'page': 0}
{'source': 'cappuccetto_rosso.pdf', 'page': 0}
{'source': 'cappuccetto_rosso.pdf', 'page': 0}
{'source': 'cappuccetto_rosso.pdf', 'page': 1}
{'source': 'cappuccetto_rosso.pdf', 'page': 1}
{'source': 'cappuccetto_rosso.pdf', 'page': 1}
{'source': 'hansel_gretel.pdf', 'page': 0}
{'source': 'hansel_gretel.pdf', 'page': 0}
{'source': 'hansel_gretel.pdf', 'page': 0}
{'source': 'hansel_gretel.pdf', 'page': 1}
{'source': 'hansel_gretel.pdf', 'page': 1}
{'source': 'hansel_gretel.pdf', 'page': 1}
{'source': 'hansel_gretel.pdf', 'page': 1}
{'source': 'hansel_gretel.pdf', 'page': 2}
{'source': 'hansel_gretel.pdf', 'page': 2}
{'source': 'hansel_gretel.pdf', 'page': 2}
{'source': 'hansel_gretel.pdf', 'page': 3}
{'source': 'hansel_gretel.pdf', 'page': 3}
{'source': 'hansel_gretel.pdf', 'page': 3}
{'source': 'hansel_gretel.pdf', 'page': 3}
{'source': 'hansel_gretel.pdf', 'page': 4}
{'source': 'hansel_gretel.pdf', 'page': 4}
{'source': 'lepre_tartaruga.pd

In [ ]:
print(f'Documents have been split into {len(docs)} pages; \
  here\'s the first one, that is from \
  {docs[1].metadata["source"]}:\n{docs[1].page_content}')

Documents have been split into 23 pages;   here's the first one, that is from   cappuccetto_rosso.pdf:
 Vino e focaccia: ieri abbiamo cotto il pane; così la nonna, che è debole e malata, se la godrà un po' e si rinforzerà. - Dove abita la tua nonna, Cappuccetto Rosso? - A un buon quarto d'ora di qui, nel bosco, sotto le tre grosse querce; là c'è la sua casa, è sotto la macchia di noccioli, lo saprai già, - disse Cappuccetto Rosso.  Il lupo pensava: " Questa bimba tenerella è un grasso boccone, sarà più saporita della vecchia; se sei furbo, le acchiappi tutt'e due". Fece un pezzetto di strada vicino a Cappuccetto Rosso, poi disse: - Vedi, Cappuccetto Rosso, quanti bei fiori? perché non ti guardi intorno? Credo che non senti neppure come cantano dolcemente gli uccellini! Te ne vai tutta contegnosa, come se andassi a scuola, ed è così allegro fuori nel bosco! Cappuccetto Rosso alzò gli occhi e quando vide i raggi di sole danzare attraverso gli alberi, e tutto intorno pieno di bei fiori, p

In [ ]:
# here is the structure of each page
docs

[Document(metadata={'source': 'cappuccetto_rosso.pdf', 'page': 0}, page_content="Cappuccetto Rosso! di Jakob e Wilhelm Grimm   C'era una volta una cara ragazzina; solo a vederla le volevan tutti bene, e specialmente la nonna, che non sapeva più cosa regalarle. Una volta le regalò un cappuccetto di velluto rosso, e, poiché le donava tanto ch'essa non volle più portare altro, la chiamarono sempre Cappuccetto Rosso. Un giorno sua madre le disse:  - Vieni, Cappuccetto Rosso, eccoti un pezzo di focaccia e una bottiglia di vino, portali alla nonna; è debole e malata e si ristorerà. Mettiti in via prima che faccia troppo caldo; e, quando sei fuori, va' da brava, senza uscir di strada; se no, cadi e rompi la bottiglia e la nonna resta a mani vuote. E quando entri nella sua stanza, non dimenticare di dir buon giorno invece di curiosare in tutti gli angoli. -Farò tutto per bene, - disse Cappuccetto Rosso alla mamma e le diede la mano.  Ma la nonna abitava fuori, nel bosco, a una mezz'ora dal vil

## RAG module

In [ ]:
!pip install -U sentence-transformers
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 50.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
from langchain_community.embeddings import (
    HuggingFaceEmbeddings
)

# given our documents, we have to make the understandable by computer
# the encoder aims at this
encoder = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L12-v2',
                                model_kwargs = {'device': "cpu"})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# for example, let's see what happens if we encode the first page of the document
embedding_page0 = encoder.embed_query(docs[0].page_content)
print(f'The embedding obtained has dimension {len(embedding_page0)}')
print(embedding_page0)

The embedding obtained has dimension 384
[-0.1149665117263794, 0.02486840821802616, 0.009546753019094467, -0.03667840361595154, -0.07811838388442993, 0.001379824010655284, 0.07318557053804398, 0.03584599494934082, 0.04024295508861542, 0.00850268267095089, 0.022586524486541748, 0.04430996999144554, 0.010790332220494747, 0.05958406627178192, -0.05406417325139046, -0.010512123815715313, 0.09719105064868927, 0.0774819627404213, 0.046332720667123795, 0.0007716019172221422, 0.04550027474761009, -0.024705467745661736, 0.028578605502843857, -0.013177316635847092, -0.04518090933561325, -0.004127667285501957, -0.03576553985476494, 0.033729758113622665, 0.002209658268839121, 0.012224624864757061, -0.026245949789881706, -0.04606025293469429, 0.018613165244460106, -0.050446271896362305, 0.00045701273484155536, -0.050471819937229156, 0.016154395416378975, -0.02960820496082306, 0.07929075509309769, 0.0063928524032235146, -0.07807956635951996, -0.03733937814831734, -0.005294439848512411, 0.03233378380

In [ ]:
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

# make all the documents treatable by the model
faiss_db = FAISS.from_documents(docs, encoder,
                                distance_strategy=DistanceStrategy.DOT_PRODUCT)

In [ ]:
def inference(question: str, context: str):

    if context == None or context == "":
        prompt = f"""Dai una risposta dettagliata alla seguente domanda. \
        Domanda: {question}"""
    else:
        prompt = f"""Usando le informazione del seguente contesto, \
        rispondi in maniera dettagliata alla seguente domanda.
            Contesto: {context}.
            Domanda: {question}"""
    chat = [
        {"role": "user", "content": prompt},
    ]
    formatted_prompt = tokenizer.apply_chat_template(
        chat,
        tokenize=False,
        add_generation_prompt=True,
    )
    inputs = tokenizer.encode(
        formatted_prompt, add_special_tokens=False, return_tensors="pt"
    ).to(device)
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=250,
            do_sample=False,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    response = response[len(formatted_prompt) :]  # remove input prompt from reponse
    response = response.replace("<|eot_id|>", "")  # remove eos token
    return response


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# given our question, we retrieve the document with the hihest probability of containing the answer
question = "Contro chi gareggia la lepre?"
retrieved_docs = faiss_db.similarity_search(question, k=1)
context = "".join(doc.page_content + "\n" for doc in retrieved_docs)

torch.cuda.empty_cache()

print(f"\n\nDomanda: {question}")
print(f"Risposta: {inference(question=question, context=context)}")
print("\nPer questa risposta ho usato il seguente documento:")
for doc in retrieved_docs:
    print(doc.metadata)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpec



Domanda: Contro chi gareggia la lepre?
Risposta: La lepre gareggia contro la tartaruga.

Per questa risposta ho usato il seguente documento:
{'source': 'lepre_tartaruga.pdf', 'page': 0}


In [ ]:
# given our question, we retrieve the document with the hihest probability of containing the answer
question = "Come si chiama la sorella di Hansel?"
retrieved_docs = faiss_db.similarity_search(question, k=1)
context = "".join(doc.page_content + "\n" for doc in retrieved_docs)

torch.cuda.empty_cache()

print(f"\n\nDomanda: {question}")
print(f"Risposta: {inference(question=question, context=context)}")
print("\nPer questa risposta ho usato il seguente documento:")
for doc in retrieved_docs:
    print(doc.metadata)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




Domanda: Come si chiama la sorella di Hansel?
Risposta: Secondo il contesto, la sorella di Hänsel si chiama Gretel.

Per questa risposta ho usato il seguente documento:
{'source': 'hansel_gretel.pdf', 'page': 1}


In [ ]:
# given our question, we retrieve the document with the hihest probability of containing the answer
question = "Come si chiama la sorella di Gretel?"
retrieved_docs = faiss_db.similarity_search(question, k=1)
context = "".join(doc.page_content + "\n" for doc in retrieved_docs)

torch.cuda.empty_cache()

print(f"\n\nDomanda: {question}")
print(f"Risposta: {inference(question=question, context=context)}")
print("\nPer questa risposta ho usato il seguente documento:")
for doc in retrieved_docs:
    print(doc.metadata)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




Domanda: Come si chiama la sorella di Gretel?
Risposta: La sorella di Gretel non è menzionata nel testo, quindi non ha un nome. Il testo parla solo di Gretel e Hänsel, fratello e sorella, ma non menziona il nome della sorella.

Per questa risposta ho usato il seguente documento:
{'source': 'hansel_gretel.pdf', 'page': 3}


In [ ]:
# given our question, we retrieve the document with the hihest probability of containing the answer
question = "In quale favola si menziona un lupo?"
retrieved_docs = faiss_db.similarity_search(question, k=2)
context = "".join(doc.page_content + "\n" for doc in retrieved_docs)

torch.cuda.empty_cache()

print(f"\n\nDomanda: {question}")
print(f"Risposta: {inference(question=question, context=context)}")
print("\nPer questa risposta ho usato il seguente documento:")
for doc in retrieved_docs:
    print(doc.metadata)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




Domanda: In quale favola si menziona un lupo?
Risposta: n due favole si menziona un lupo: in "Cappuccetto Rosso" (dove il lupo è il mostro che cerca di mangiare la bambina) e in "Hänsel e Gretel" (dove non è il mostro che cerca di mangiare i bambini, ma bussando alla porta, cerca di ingannarli).

Per questa risposta ho usato il seguente documento:
{'source': 'cappuccetto_rosso.pdf', 'page': 1}
{'source': 'hansel_gretel.pdf', 'page': 1}


## Let's build an APP with Gradio

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 35

In [ ]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(fn=greet, inputs="text", outputs="text")
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://75dc828f6d9f52305c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr

# given a question, we use RAG module to retrieve the document, and the gradio app shows the answer
def llm_rag(question):

  # retrieve the k=1 documents with the hihest probability of containing the answer
  retrieved_docs = faiss_db.similarity_search(question, k=1)

  # concatenate all the pages of the documents to provide the context
  context = "".join(doc.page_content + "\n" for doc in retrieved_docs)

  # clear the cache
  torch.cuda.empty_cache()

  # print the answer
  answer = f"{inference(question=question, context=context)}" + \
    "\n\nPer questa risposta ho usato il seguente documento:\n" + \
    "\n".join(str(doc.metadata) for doc in retrieved_docs)

  return answer

# launch the gradio app
demo = gr.Interface(fn=llm_rag, inputs="text", outputs="text")
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://de5f1903d37689d228.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
